# IA25_P01_G06 — Project 01: Class Timetable (CSP)


# 1. Introduction
Creating academic schedules is a complex challenge for higher education institutions. At IPCA, this manual process consumes significant resources and is prone to errors. This project aims to develop an automated solution based on Artificial Intelligence techniques to generate optimized schedules that satisfy all academic constraints.

**Objectives:**

The objectives are to formulate the schedule problem for an institution, implement hard and soft constraints, and generate a schedule for the dataset provided.

**Teammates:**

- Ana Paula
- Daniel
- Eva
- Glória
- Thiago Yabuki de Araujo - a24207



# 2. Agent design (CSP)

We model each lesson (class–course–lesson index) as a variable whose domain is the set of timeslot indices (1..20). Hard constraints include:
- A class cannot have two lessons at the same timeslot (AllDifferent per class).
- A teacher cannot teach two lessons at the same timeslot (AllDifferent per teacher).
- Teacher availability restrictions (prohibited timeslots).
- Maximum 3 lessons per day per class.
- Online lessons constraint (if required, both online lessons scheduled the same day).


## Variables and Domain
- Each variable represents a specific class: {turma}_{curso}_aula{índice}
- Domain: Timeslots 1 a 20 (5 dias × 4 horários/dia)


In [89]:
from constraint import Problem, AllDifferentConstraint

def define_variables(problem, dados):
    problem = Problem()
    dominio_timeslots = list(range(1, 21))
    variables = []
    for turma, cursos in dados['cc'].items():
            for curso in cursos:
                for aula_index in [1, 2]:
                    var_name = f"{turma}_{curso}_aula{aula_index}"
                    problem.addVariable(var_name, dominio_timeslots)
                    variables.append(var_name)

    print(f"Total de aulas para agendar: {len(variables)}")

## Hard Constraints

In [90]:
from constraint import AllDifferentConstraint

dias_semana = ["Segunda", "Terça", "Quarta", "Quinta", "Sexta"]
horarios = ["09h-11h", "11h-13h", "14h-16h", "16h-18h"]

BLOCOS_POR_DIA = len(horarios)
DIAS_SEMANA = len(dias_semana)
TOTAL_TIMESLOTS = BLOCOS_POR_DIA * DIAS_SEMANA
BLOCKS_PER_DAY = BLOCOS_POR_DIA 

def constraints(problem, dados):
    variables = []
    # RESTRIÇÕES (HARD CONSTRAINTS)
    
    # Aulas diferentes por turma (ALLDIFF)
    for turma in dados['cc'].keys():
        turma_vars = [var for var in variables if var.startswith(f"{turma}_")]
        problem.addConstraint(AllDifferentConstraint(), turma_vars)

    # Docente não pode dar duas aulas ao mesmo tempo
    for professor, cursos_prof in dados['dsd'].items():
        professor_vars = []
        for var_name in variables:
            curso = var_name.split('_')[1]
            if curso in cursos_prof:
                professor_vars.append(var_name)
        if professor_vars:
            problem.addConstraint(AllDifferentConstraint(), professor_vars)

    # Restrições de disponibilidade do Docente
    for professor, slots_indisponiveis in dados['tr'].items():
        cursos_prof = dados['dsd'][professor]
        for var_name in variables:
            curso = var_name.split('_')[1]
            if curso in cursos_prof:
                def professor_disponivel(timeslot, indisponiveis=slots_indisponiveis):
                    return timeslot not in indisponiveis

                problem.addConstraint(professor_disponivel, [var_name])

    # Máximo 3 aulas por dia por turma
    for turma in dados['cc'].keys():
        turma_vars = [var for var in variables if var.startswith(f"{turma}_")]

        for dia in range(DIAS_SEMANA):
            slots_do_dia = list(range(dia * BLOCOS_POR_DIA + 1, (dia + 1) * BLOCOS_POR_DIA + 1))

            # Constraint individual para cada dia
            def max_aulas_dia(*assignments, slots_dia=slots_do_dia):
                count = 0
                for slot in assignments:
                    if slot in slots_dia:
                        count += 1
                return count <= 3

            problem.addConstraint(max_aulas_dia, turma_vars)

    # Aulas online ao mesmo dia
    for curso_online in dados['oc'].keys():
        aulas_online = [var for var in variables if f"_{curso_online}_" in var]
        if len(aulas_online) >= 2:
            def aulas_online_mesmo_dia(timeslot1, timeslot2):
                dia1 = (timeslot1 - 1) // BLOCOS_POR_DIA
                dia2 = (timeslot2 - 1) // BLOCOS_POR_DIA
                return dia1 == dia2
            problem.addConstraint(aulas_online_mesmo_dia, aulas_online)
            

## Functions Files

### dataset.txt

In [91]:
dataset_content = r'''#head
— All classes last 2 hours
— Classes will be schedule from Monday to Friday
— Each day has 4 blocks of 2 hour each
— Blocks are numbered from 1 to 20 (5 days *4 blocks), Monday 9am to Friday 4pm 
— In this dataset alll classes have 2 lessons per week

#cc
t01         UC11 UC12 UC13 UC14 UC15
t02         UC21 UC22 UC23 UC24 UC25
t03         UC31 UC32 UC33 UC34 UC35

#olw

#dsd
jo          UC11 UC21 UC22 UC31
mike     UC12 UC23 UC32
rob        UC13 UC14 UC24 UC33
sue       UC15 UC25 UC34 UC35

#tr
mike   13 14 15 16 17 18 19 20
rob      1 2 3 4
sue      9 10 11 12 17 18 19 20

#rr
UC14   Lab01
UC22   Lab01

#oc
UC21   2
UC31   2
'''
open('dataset.txt','w',encoding='utf-8').write(dataset_content)
print('dataset.txt written to notebook working directory')
print('\n--- dataset head:\n')
print('\n'.join(dataset_content.splitlines()[:30]))


dataset.txt written to notebook working directory

--- dataset head:

#head
— All classes last 2 hours
— Classes will be schedule from Monday to Friday
— Each day has 4 blocks of 2 hour each
— Blocks are numbered from 1 to 20 (5 days *4 blocks), Monday 9am to Friday 4pm 
— In this dataset alll classes have 2 lessons per week

#cc
t01         UC11 UC12 UC13 UC14 UC15
t02         UC21 UC22 UC23 UC24 UC25
t03         UC31 UC32 UC33 UC34 UC35

#olw

#dsd
jo          UC11 UC21 UC22 UC31
mike     UC12 UC23 UC32
rob        UC13 UC14 UC24 UC33
sue       UC15 UC25 UC34 UC35

#tr
mike   13 14 15 16 17 18 19 20
rob      1 2 3 4
sue      9 10 11 12 17 18 19 20

#rr
UC14   Lab01
UC22   Lab01

#oc


### parser.py

In [92]:
def ler_ficheiro(nome="dataset.txt"):
    dados = {
        'cc': {},   # Cursos por turma
        'olw': [],  # Cursos com uma aula por semana
        'dsd': {},  # Cursos por professor
        'tr': {},   # Restrições de horário dos professores
        'rr': {},   # Cursos e respetivas salas
        'oc': {}    # Cursos e respetivos índices de aula
    }
    try:
        with open(nome, 'r', encoding='utf-8') as ficheiro:
            linhas = ficheiro.readlines()

        seccao_atual = None

        for linha in linhas:
            linha = linha.strip()

            # Ignorar linhas vazias
            if not linha:
                continue

            # Detetar secções
            if linha.startswith('#cc'):
                seccao_atual = 'cc'
                continue
            elif linha.startswith('#olw'):
                seccao_atual = 'olw'
                continue
            elif linha.startswith('#dsd'):
                seccao_atual = 'dsd'
                continue
            elif linha.startswith('#tr'):
                seccao_atual = 'tr'
                continue
            elif linha.startswith('#rr'):
                seccao_atual = 'rr'
                continue
            elif linha.startswith('#oc'):
                seccao_atual = 'oc'
                continue
            elif linha.startswith('#head'):
                seccao_atual = None
                continue

            # Processar dados baseado na secção atual
            if seccao_atual == 'cc':
                partes = linha.split()
                if partes:
                    turma = partes[0]
                    cursos = partes[1:]
                    dados['cc'][turma] = cursos

            elif seccao_atual == 'olw':
                # No exemplo está vazio, mas processamos caso existam dados
                if linha and not linha.startswith('#'):
                    cursos = linha.split()
                    dados['olw'].extend(cursos)

            elif seccao_atual == 'dsd':
                partes = linha.split()
                if partes:
                    professor = partes[0]
                    cursos = partes[1:]
                    dados['dsd'][professor] = cursos

            elif seccao_atual == 'tr':
                partes = linha.split()
                if partes:
                    professor = partes[0]
                    slots = list(map(int, partes[1:]))
                    dados['tr'][professor] = slots

            elif seccao_atual == 'rr':
                partes = linha.split()
                if len(partes) >= 2:
                    curso = partes[0]
                    sala = partes[1]
                    dados['rr'][curso] = sala

            elif seccao_atual == 'oc':
                partes = linha.split()
                if len(partes) >= 2:
                    curso = partes[0]
                    indice_aula = int(partes[1])
                    dados['oc'][curso] = indice_aula

        print(f"✅ File '{nome}' read successfully!")
        return dados

    except FileNotFoundError:
        print(f"Error: File '{nome}' not found!")
        return None
    except Exception as e:
        print(f"Error reading file: {e}")
        return None

def mostrar_dados(dados):
    if not dados:
        print("No data to show!")
        return

    # Mostrar cursos por turma (cc)
    print("\nCOURSES BY CLASS (#cc):")
    print("-" * 40)
    for turma, cursos in dados['cc'].items():
        print(f"{turma}: {', '.join(cursos)}")
    print("=" * 50)

    # Mostrar cursos com uma aula por semana (olw)
    print(f"\nCOURSES WITH ONE LESSON PER WEEK (#olw):")
    print("-" * 40)
    if dados['olw']:
        print(', '.join(dados['olw']))
    else:
        print("No course with one lesson per week")
    print("=" * 50)

    # Mostrar cursos por professor (dsd)
    print("\nCOURSES BY TEACHER (#dsd):")
    print("-" * 40)
    for professor, cursos in dados['dsd'].items():
        print(f"{professor}: {', '.join(cursos)}")
    print("=" * 50)

    # Mostrar restrições de horário (tr)
    print("\nTEACHER TIMESLOT RESTRICTIONS (#tr):")
    print("-" * 40)
    for professor, slots in dados['tr'].items():
        slots_str = ', '.join(map(str, slots))
        print(f"{professor}: slots {slots_str}")
    print("=" * 50)

    # Mostrar restrições de sala (rr)
    print("\nROOM RESTRICTIONS (#rr):")
    print("-" * 40)
    for curso, sala in dados['rr'].items():
        print(f"{curso}: {sala}")

    print("=" * 50)
    # Mostrar aulas online (oc)
    print("\nONLINE LESSONS (#oc):")
    print("-" * 40)
    for curso, indice in dados['oc'].items():
        print(f"{curso}: lesson {indice} is online")


### model.py

In [93]:
from constraint import Problem, AllDifferentConstraint

dias_semana = ["Segunda", "Terça", "Quarta", "Quinta", "Sexta"]
horarios = ["09h-11h", "11h-13h", "14h-16h", "16h-18h"]

BLOCOS_POR_DIA = len(horarios)
DIAS_SEMANA = len(dias_semana)
TOTAL_TIMESLOTS = BLOCOS_POR_DIA * DIAS_SEMANA
BLOCKS_PER_DAY = BLOCOS_POR_DIA 

# Função para criar o quadro de horários vazio
def criar_quadro():
    quadro = []
    for i in range(BLOCOS_POR_DIA):
        linha = []
        for j in range(DIAS_SEMANA):
            timeslot_index = j * BLOCOS_POR_DIA + i + 1
            linha.append(f"timeslot_{timeslot_index}")
        quadro.append(linha)
    return quadro   

# Função para desenhar o quadro no terminal
def visualizar_quadro(quadro):
    cabecalho = f"{'':<12} | "
    for dia in dias_semana:
        cabecalho += f"{dia:<15} | "
    print(cabecalho)
    print("-" * 110)

    for i in range(BLOCOS_POR_DIA):
        linha_str = f"{horarios[i]:<12} | "
        for j in range(DIAS_SEMANA):
            timeslot = quadro[i][j]
            linha_str += f"{timeslot:<15} | "
        print(linha_str)
        print("-" * 110)

### util.py

In [94]:
from constraint import Problem, AllDifferentConstraint

def criar_quadro_horario_com_aulas():
    # Cria um quadro horário com a estrutura para as aulas
    quadro = []
    for i in range(BLOCOS_POR_DIA):
        linha = []
        for j in range(DIAS_SEMANA):
            timeslot_index = j * BLOCOS_POR_DIA + i + 1
            linha.append({
                'timeslot': f"timeslot_{timeslot_index}",
                'aulas': [],
                'dia': dias_semana[j],
                'horario': horarios[i],
                'numero': timeslot_index
            })
        quadro.append(linha)
    return quadro


def atribuir_aulas_ao_horario(dados):
    # Atribui aulas seguindo e respeitando restrições
    problem = Problem()
    variables = []
    dominio_timeslots = list(range(1, TOTAL_TIMESLOTS + 1))

    # Cria variáveis
    for turma, cursos in dados['cc'].items():
        for curso in cursos:
            for aula_index in [1, 2]:
                var_name = f"{turma}_{curso}_aula{aula_index}"
                problem.addVariable(var_name, dominio_timeslots)
                variables.append(var_name)

    print(f"Total de aulas para agendar: {len(variables)}")

    # RESTRIÇÕES (HARD CONSTRAINTS)
    
    # Aulas diferentes por turma (ALLDIFF)
    for turma in dados['cc'].keys():
        turma_vars = [var for var in variables if var.startswith(f"{turma}_")]
        problem.addConstraint(AllDifferentConstraint(), turma_vars)

    # Docente não pode dar duas aulas ao mesmo tempo
    for professor, cursos_prof in dados['dsd'].items():
        professor_vars = []
        for var_name in variables:
            curso = var_name.split('_')[1]
            if curso in cursos_prof:
                professor_vars.append(var_name)
        if professor_vars:
            problem.addConstraint(AllDifferentConstraint(), professor_vars)

    # Restrições de disponibilidade do Docente
    for professor, slots_indisponiveis in dados['tr'].items():
        cursos_prof = dados['dsd'][professor]
        for var_name in variables:
            curso = var_name.split('_')[1]
            if curso in cursos_prof:
                def professor_disponivel(timeslot, indisponiveis=slots_indisponiveis):
                    return timeslot not in indisponiveis

                problem.addConstraint(professor_disponivel, [var_name])

    # Máximo 3 aulas por dia por turma
    for turma in dados['cc'].keys():
        turma_vars = [var for var in variables if var.startswith(f"{turma}_")]

        for dia in range(DIAS_SEMANA):
            slots_do_dia = list(range(dia * BLOCOS_POR_DIA + 1, (dia + 1) * BLOCOS_POR_DIA + 1))

            # Constraint individual para cada dia
            def max_aulas_dia(*assignments, slots_dia=slots_do_dia):
                count = 0
                for slot in assignments:
                    if slot in slots_dia:
                        count += 1
                return count <= 3

            problem.addConstraint(max_aulas_dia, turma_vars)


    #TENTATIVA ANTERIOR
    # for turma in dados['cc'].keys():
    #     turma_vars = [var for var in variables if var.startswith(f"{turma}_")]
    #     for dia in range(DIAS_SEMANA):
    #         slots_do_dia = list(range(dia * BLOCOS_POR_DIA + 1, (dia + 1) * BLOCOS_POR_DIA + 1))
    #
    #         def max_3_aulas_por_dia(*assignments):
    #             aulas_no_dia = 0
    #             for timeslot in assignments:
    #                 if timeslot in slots_do_dia:
    #                     aulas_no_dia += 1
    #             return aulas_no_dia <= 3
    #
    #         problem.addConstraint(max_3_aulas_por_dia, turma_vars)


    # Aulas online ao mesmo dia
    for curso_online in dados['oc'].keys():
        aulas_online = [var for var in variables if f"_{curso_online}_" in var]
        if len(aulas_online) >= 2:
            def aulas_online_mesmo_dia(timeslot1, timeslot2):
                dia1 = (timeslot1 - 1) // BLOCOS_POR_DIA
                dia2 = (timeslot2 - 1) // BLOCOS_POR_DIA
                return dia1 == dia2
            problem.addConstraint(aulas_online_mesmo_dia, aulas_online)

    # for curso, aula_online_index in dados['oc'].items():
    #         aulas_curso = [var for var in variables if f"_{curso}_" in var]
    #         if len(aulas_curso) >= 2:
    #             def aulas_mesmo_dia(*timeslots):
    #                 dias = [(ts - 1) // BLOCOS_POR_DIA for ts in timeslots]
    #                 return all(dia == dias[0] for dia in dias)
    #           problem.addConstraint(aulas_mesmo_dia, aulas_curso)
            
    print("🔍 A resolver o problema de agendamento...")
    solution = problem.getSolution()

    if not solution:
        print("❌ Não foi possível encontrar uma solução!")
        return None

    print("✅ Solução encontrada!")
    return solution

def preencher_quadro_com_solucao(quadro, solution, dados):
    # Preenche o quadro horário com a solução encontrada
    for var_name, timeslot in solution.items():
        partes = var_name.split('_')
        turma = partes[0]
        curso = partes[1]

        timeslot_index = timeslot - 1

        linha = timeslot_index % BLOCOS_POR_DIA  # 0-3
        coluna = timeslot_index // BLOCOS_POR_DIA  # 0-4

        sala = dados['rr'].get(curso, f"Room{turma[-1]}")
        aula_info = {'turma': turma, 'curso': curso, 'sala': sala}
        quadro[linha][coluna]['aulas'].append(aula_info)

    return quadro


def visualizar_horario_por_turma(quadro_geral, dados):
    # Mostra horário de cada turma
    for turma in dados['cc'].keys():
        print(f"\n" + "=" * 80)
        print(f"HORÁRIO DA TURMA {turma}")
        print("=" * 80)

        cabecalho = f"{'Horário':<12} |"
        for dia in dias_semana:
            cabecalho += f" {dia:<20} |"
        print(cabecalho)
        print("-" * 140)

        for i in range(BLOCOS_POR_DIA):
            linha_str = f"{horarios[i]:<12} |"

            for j in range(DIAS_SEMANA):
                celula = quadro_geral[i][j]
                conteudo = "---"

                for aula in celula['aulas']:
                    if aula['turma'] == turma:
                        conteudo = f"{aula['curso']} {aula['sala']}"
                        break

                linha_str += f" {conteudo:<20} |"

            print(linha_str)
            print("-" * 140)


def verificar_restricoes(quadro, dados):
    # Verifica se todas as restrições foram cumpridas
    print("\n" + "🔍 VERIFICAÇÃO DE RESTRIÇÕES")
    print("=" * 50)

    problemas = []

    # Verificar conflitos de professor
    for i in range(BLOCOS_POR_DIA):
        for j in range(DIAS_SEMANA):
            celula = quadro[i][j]
            professores_na_celula = {}

            for aula in celula['aulas']:
                # Busca o professor da aula atribuída
                professor = None
                for prof, cursos in dados['dsd'].items():
                    if aula['curso'] in cursos:
                        professor = prof
                        break

                if professor:
                    if professor in professores_na_celula:
                        problemas.append(
                            f"❌ Professor {professor} com aula dupla: {professores_na_celula[professor]} e {aula['curso']} no {celula['dia']} {celula['horario']}")
                    professores_na_celula[professor] = aula['curso']

    # Verificar restrições de horário do professor
    for professor, slots_proibidos in dados['tr'].items():
        for timeslot_proibido in slots_proibidos:
            coluna = (timeslot_proibido - 1) // BLOCOS_POR_DIA
            linha = (timeslot_proibido - 1) % BLOCOS_POR_DIA

            celula = quadro[linha][coluna]
            for aula in celula['aulas']:
                if aula['curso'] in dados['dsd'].get(professor, []):
                    problemas.append(
                        f"❌ Professor {professor} a dar {aula['curso']} em slot proibido {timeslot_proibido} ({celula['dia']} {celula['horario']})")

    # Verificar máximo 3 aulas por dia por turma
    for turma in dados['cc'].keys():
        for dia_idx in range(DIAS_SEMANA):
            aulas_no_dia = 0
            for i in range(BLOCOS_POR_DIA):
                celula = quadro[i][dia_idx]
                for aula in celula['aulas']:
                    if aula['turma'] == turma:
                        aulas_no_dia += 1

            if aulas_no_dia > 3:
                problemas.append(f"❌ Turma {turma} com {aulas_no_dia} aulas na {dias_semana[dia_idx]} (máximo: 3)")

    # Apresenta resultados
    if problemas:
        print("⛔ PROBLEMAS ENCONTRADOS:")
        for problema in problemas:
            print(problema)
    else:
        print("✅ TODAS AS RESTRIÇÕES CUMPRIDAS!")

    return len(problemas) == 0

       
def agendar_horario(dados):
    print("INICIANDO AGENDAMENTO")

    # Cria quadro vazio
    quadro = criar_quadro_horario_com_aulas()

    # Atribui aulas
    solution = atribuir_aulas_ao_horario(dados)
    if not solution:
        print("Não foi possível criar o horário.")
        return

    # Preenche quadro
    quadro_preenchido = preencher_quadro_com_solucao(quadro, solution, dados)

    # Verifica restrições
    todas_cumpridas = verificar_restricoes(quadro_preenchido, dados)
    print(f"\nVerificação final das restrições: {'Todas cumpridas' if todas_cumpridas else 'Problemas encontrados'}")

    # Mostra horários
    print("\n" + "HORÁRIOS FINAIS")
    print("=" * 50)
    visualizar_horario_por_turma(quadro_preenchido, dados)

### main.py

In [95]:
# Main script adapted for the notebook environment (text-based display)
from constraint import Problem, AllDifferentConstraint

def main(dados):
    print("=" * 50)
    print("SCHEDULE MANAGEMENT SYSTEM")
    print("=" * 50)

    # 1. Create and show empty timetable frame
    quadro = criar_quadro()
    print("\n BASE TIMETABLE: \n")
    visualizar_quadro(quadro)

    # 2. Use provided data (dados argument)
    if dados:
       # 3. Show loaded data
       mostrar_dados(dados)
       print("\n" + "STARTING AUTOMATIC SCHEDULING")
       print("=" * 50)
       agendar_horario(dados)
    else:
        print("Could not load data.")


### Running main.py

In [96]:
# Load dataset via parser.ler_ficheiro and run the notebook_main which calls the embedded agendar_horario
dados = ler_ficheiro('dataset.txt')
main(dados)

✅ File 'dataset.txt' read successfully!
SCHEDULE MANAGEMENT SYSTEM

 BASE TIMETABLE: 

             | Segunda         | Terça           | Quarta          | Quinta          | Sexta           | 
--------------------------------------------------------------------------------------------------------------
09h-11h      | timeslot_1      | timeslot_5      | timeslot_9      | timeslot_13     | timeslot_17     | 
--------------------------------------------------------------------------------------------------------------
11h-13h      | timeslot_2      | timeslot_6      | timeslot_10     | timeslot_14     | timeslot_18     | 
--------------------------------------------------------------------------------------------------------------
14h-16h      | timeslot_3      | timeslot_7      | timeslot_11     | timeslot_15     | timeslot_19     | 
--------------------------------------------------------------------------------------------------------------
16h-18h      | timeslot_4      | timeslot_8  

# Agent Running


# Conclusion